# Requirements

In [1]:
import os
import json
import pandas as pd
import numpy as np
import json
import pickle
import random
import unicodedata
import requests
import re
import pyconll # universal dependencies parser
import spacy
import io

import sddk

In [2]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [28]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_dated_20201126.json", "df", conf)
# save locally for the next time
#AGT.to_json("../data/large_files/AGT.json")

In [29]:
AGT.head(5)

,filename,author,title,string,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...",38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,Epici/-ae
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,\nΘουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶ...,150126,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Historici/-ae
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,110773,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,Biographi
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,\n̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα...,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici


In [30]:
# character count
AGT_string = " ".join(AGT["string"].tolist())
len(AGT_string)

219532769

# Preprocessing

In [31]:
# short messy string for testing
test_string = "ἀρχόμενος ³σέο, <Φοῖβε>, text in latin also CAPITAL     ⁴παλαιγενέων 1234 κλέα 34056 φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ. τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδʼ ἀνέρος, ὅντινʼ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπʼ ἐννεσίῃσι δαμῆναι."
test_string

'ἀρχόμενος ³σέο, <Φοῖβε>, text in latin also CAPITAL     ⁴παλαιγενέων 1234 κλέα 34056 φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ. τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδʼ ἀνέρος, ὅντινʼ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπʼ ἐννεσίῃσι δαμῆναι.'

In [32]:
# coptic "c"
string = 'ἀρχόμενος σέο, Φοῖβεϲ, παλαιγενϲέων κλέαϲ φωτῶν μνήσομαι,'
string = re.sub("ϲ(\W)", r"ς\1", string) # if "ϲ" is last letter of a word
string = re.sub("ϲ(\w)", r"σ\1", string) # if "ϲ" is within a word
string

'ἀρχόμενος σέο, Φοῖβες, παλαιγενσέων κλέας φωτῶν μνήσομαι,'

In [41]:
"’" == "ʼ"

False

In [33]:
# remove numerical superscripts
def remove_sups(string):
    sups = ["¹", "²", "³", "⁴","⁵", "⁶", "⁷", "⁰", "⁹"]
    for sup in sups:
        string = string.replace(sup, "")
    return string
def normalize_encoding(string):
    return unicodedata.normalize("NFC", string)

v_ud = "’"
v_agt = "ʼ"

def clean_string(string):
    string = normalize_encoding(string)
    string = remove_sups(string)
    string = re.sub("[a-zA-Z0-9]", "", string)
    string = re.sub("<|>", "", string)
    string = re.sub("ϲ(\W)", r"ς\1", string) # if coptic "ϲ" is last letter of a word
    string = re.sub("ϲ(\w)", r"σ\1", string) # if coptic "ϲ" is within a word
    string = re.sub(v_ud, v_agt, string)
    string = re.sub("\s+", " ", string)
    return string

In [34]:
# checking how it works
clean_string(test_string)

'ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ. τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδʼ ἀνέρος, ὅντινʼ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπʼ ἐννεσίῃσι δαμῆναι.'

In [35]:
%%time
# new column for clean text
AGT["clean_string"] = AGT["string"].apply(clean_string) # locally it took me 42.9 s

CPU times: user 24.1 s, sys: 721 ms, total: 24.8 s
Wall time: 24.8 s


In [36]:
# original wordcount before cleaning
AGT["wordcount"].sum() 

33419267

In [37]:
# updated wordcount based on cleaned string
AGT["wordcount"] = AGT["clean_string"].apply(lambda x: len(x.split()))
AGT["wordcount"].sum()

31248866

In [38]:
AGT.drop("string", axis=1, inplace=True)

# Generate sentences

In [39]:
# clean string consisting of a couple of sentences
string = "'ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ. τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδʼ ἀνέρος, ὅντινʼ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπʼ ἐννεσίῃσι δαμῆναι. δηρὸν δʼ οὐ μετέπειτα τεὴν κατὰ βάξιν Ἰήσων χειμερίοιο ῥέεθρα κιὼν διὰ ποσσὶν Ἀναύρου ἄλλο μὲν ἐξεσάωσεν ὑπʼ ἰλύος, ἄλλο δʼ ἔνερθεν κάλλιπεν αὖθι πέδιλον ἐνισχόμενον προχοῇσιν. ἵκετο δʼ ἐς Πελίην αὐτοσχεδὸν ἀντιβολήσων εἰλαπίνης, ἣν πατρὶ Ποσειδάωνι καὶ ἄλλοις ῥέζε θεοῖς, Ἥρης δὲ Πελασγίδος οὐκ ἀλέγιζεν. αἶψα δὲ τόνγʼ ἐσιδὼν ἐφράσσατο, καί οἱ ἄεθλον ἔντυε ναυτιλίης πολυκηδέος, ὄφρʼ ἐνὶ πόντῳ ἠὲ καὶ ἀλλοδαποῖσι μετʼ ἀνδράσι νόστον ὀλέσσῃ. νῆα μὲν οὖν οἱ πρόσθεν ἐπικλείουσιν ἀοιδοὶ Ἄργον Ἀθηναίης καμέειν ὑποθημοσύνῃσιν. νῦν δʼ ἂν ἐγὼ γενεήν τε καὶ οὔνομα μυθησαίμην ἡρώων, δολιχῆς τε πόρους ἁλός, ὅσσα τʼ ἔρεξαν πλαζόμενοι· Μοῦσαι δʼ ὑποφήτορες εἶεν ἀοιδῆς. ἀοιδῆς"
print(string)

'ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ. τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδʼ ἀνέρος, ὅντινʼ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπʼ ἐννεσίῃσι δαμῆναι. δηρὸν δʼ οὐ μετέπειτα τεὴν κατὰ βάξιν Ἰήσων χειμερίοιο ῥέεθρα κιὼν διὰ ποσσὶν Ἀναύρου ἄλλο μὲν ἐξεσάωσεν ὑπʼ ἰλύος, ἄλλο δʼ ἔνερθεν κάλλιπεν αὖθι πέδιλον ἐνισχόμενον προχοῇσιν. ἵκετο δʼ ἐς Πελίην αὐτοσχεδὸν ἀντιβολήσων εἰλαπίνης, ἣν πατρὶ Ποσειδάωνι καὶ ἄλλοις ῥέζε θεοῖς, Ἥρης δὲ Πελασγίδος οὐκ ἀλέγιζεν. αἶψα δὲ τόνγʼ ἐσιδὼν ἐφράσσατο, καί οἱ ἄεθλον ἔντυε ναυτιλίης πολυκηδέος, ὄφρʼ ἐνὶ πόντῳ ἠὲ καὶ ἀλλοδαποῖσι μετʼ ἀνδράσι νόστον ὀλέσσῃ. νῆα μὲν οὖν οἱ πρόσθεν ἐπικλείουσιν ἀοιδοὶ Ἄργον Ἀθηναίης καμέειν ὑποθημοσύνῃσιν. νῦν δʼ ἂν ἐγὼ γενεήν τε καὶ οὔνομα μυθησαίμην ἡρώων, δολιχῆς τε πόρους ἁλός, ὅσσα τʼ ἔρεξαν πλαζόμενοι· Μοῦσαι δʼ ὑποφήτορες εἶεν ἀοιδῆς. ἀοιδῆς


In [40]:
sentence = "'ἀρχόμενος σέο*, Φοῖβε----, παλαιγενέων κλέα φωτῶν μνήσομαι,"
sentence = "".join([c for c in sentence if re.match("[\w|\s|\.|\,|\·|\:|\;|\?|\!]", c)]) 
sentence

'ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μνήσομαι,'

In [42]:
# add column with sentences
def get_sentences(string):
    splits = re.split(r"(\.|\·|\:|\;|\?|\!)" , string)
    sentences = []
    for n in range(0, len(splits), 2):
        try:
            sentence = splits[n]+splits[n+1]
            sentence = "".join([c for c in sentence if re.match("[\w|\s|\.|\,|\·|\:|\;|\?|\!]", c)]) 
            sentence = re.sub("^\s*", "", sentence)
            sentence = re.sub(v_ud, v_agt, sentence)
            sentences.append(sentence)
        except:
            sentence = splits[n]
            sentence = re.sub("^\s*", "", sentence)
            if len(sentence) > 0:
                sentences.append(sentence)
            break
    return sentences
get_sentences(string)

['ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ.',
 'τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδʼ ἀνέρος, ὅντινʼ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπʼ ἐννεσίῃσι δαμῆναι.',
 'δηρὸν δʼ οὐ μετέπειτα τεὴν κατὰ βάξιν Ἰήσων χειμερίοιο ῥέεθρα κιὼν διὰ ποσσὶν Ἀναύρου ἄλλο μὲν ἐξεσάωσεν ὑπʼ ἰλύος, ἄλλο δʼ ἔνερθεν κάλλιπεν αὖθι πέδιλον ἐνισχόμενον προχοῇσιν.',
 'ἵκετο δʼ ἐς Πελίην αὐτοσχεδὸν ἀντιβολήσων εἰλαπίνης, ἣν πατρὶ Ποσειδάωνι καὶ ἄλλοις ῥέζε θεοῖς, Ἥρης δὲ Πελασγίδος οὐκ ἀλέγιζεν.',
 'αἶψα δὲ τόνγʼ ἐσιδὼν ἐφράσσατο, καί οἱ ἄεθλον ἔντυε ναυτιλίης πολυκηδέος, ὄφρʼ ἐνὶ πόντῳ ἠὲ καὶ ἀλλοδαποῖσι μετʼ ἀνδράσι νόστον ὀλέσσῃ.',
 'νῆα μὲν οὖν οἱ πρόσθεν ἐπικλείουσιν ἀοιδοὶ Ἄργον Ἀθηναίης καμέειν ὑποθημοσύνῃσιν.',
 'νῦν δʼ ἂν ἐγὼ γενεήν τε καὶ οὔνομα μυθησαίμην ἡρώων, δολιχῆς τε πόρους ἁλός, ὅσσα τʼ ἔρεξαν πλαζόμενοι·',
 'Μοῦσαι δʼ ὑποφήτορες εἶεν ἀοιδῆς.',
 'ἀ

In [43]:
%%time
AGT["sentences"] = AGT["clean_string"].apply(get_sentences)

CPU times: user 1min 42s, sys: 159 ms, total: 1min 42s
Wall time: 1min 42s


In [44]:
AGT["n_sentences"] = AGT["sentences"].apply(lambda x: len(x))

In [45]:
# total number of sentences
AGT["n_sentences"].sum()

2891346

In [46]:
sddk.write_file("SDAM_data/AGT/AGT_preprocessed_20201209.json", AGT, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/AGT/AGT_preprocessed_20201209.json"


# Word embeddings

We turn to generate a word2vec model upon the raw text of the whole corpus. We used this vectors later on to improve performance of our parser.

In [47]:
def get_flat_sentences(series):
    sentences_list = [sent for doc in series.tolist() for sent in doc]
    return sentences_list

In [48]:
all_sentences = get_flat_sentences(AGT["sentences"])
len(all_sentences)

2891346

In [49]:
print(all_sentences[:2])

['ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μνήσομαι, οἳ Πόντοιο κατὰ στόμα καὶ διὰ πέτρας Κυανέας βασιλῆος ἐφημοσύνῃ Πελίαο χρύσειον μετὰ κῶας ἐύζυγον ἤλασαν Ἀργώ.', 'τοίην γὰρ Πελίης φάτιν ἔκλυεν, ὥς μιν ὀπίσσω μοῖρα μένει στυγερή, τοῦδʼ ἀνέρος, ὅντινʼ ἴδοιτο δημόθεν οἰοπέδιλον, ὑπʼ ἐννεσίῃσι δαμῆναι.']


In [50]:
sentence = "νῦν δʼ ἂν ἐγὼ γενεήν τε καὶ οὔνομα μυθησαίμην ἡρώων, δολιχῆς τε πόρους ἁλός, ὅσσα τʼ ἔρεξαν πλαζόμενοι·"

In [51]:
def grc_tokenization(string):
    tokenized_string = [w for w in re.split(r"(\s|\W)" , string) if w not in [" ", ""]]
    return tokenized_string

In [52]:
print(grc_tokenization(sentence))

['νῦν', 'δʼ', 'ἂν', 'ἐγὼ', 'γενεήν', 'τε', 'καὶ', 'οὔνομα', 'μυθησαίμην', 'ἡρώων', ',', 'δολιχῆς', 'τε', 'πόρους', 'ἁλός', ',', 'ὅσσα', 'τʼ', 'ἔρεξαν', 'πλαζόμενοι', '·']


In [53]:
sents_tok = [grc_tokenization(sent) for sent in all_sentences]

In [54]:
print(sents_tok[:2])

[['ἀρχόμενος', 'σέο', ',', 'Φοῖβε', ',', 'παλαιγενέων', 'κλέα', 'φωτῶν', 'μνήσομαι', ',', 'οἳ', 'Πόντοιο', 'κατὰ', 'στόμα', 'καὶ', 'διὰ', 'πέτρας', 'Κυανέας', 'βασιλῆος', 'ἐφημοσύνῃ', 'Πελίαο', 'χρύσειον', 'μετὰ', 'κῶας', 'ἐύζυγον', 'ἤλασαν', 'Ἀργώ', '.'], ['τοίην', 'γὰρ', 'Πελίης', 'φάτιν', 'ἔκλυεν', ',', 'ὥς', 'μιν', 'ὀπίσσω', 'μοῖρα', 'μένει', 'στυγερή', ',', 'τοῦδʼ', 'ἀνέρος', ',', 'ὅντινʼ', 'ἴδοιτο', 'δημόθεν', 'οἰοπέδιλον', ',', 'ὑπʼ', 'ἐννεσίῃσι', 'δαμῆναι', '.']]


In [55]:
# based on this tutorial: https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

In [56]:
import multiprocessing
from gensim.models import Word2Vec
from time import time  # To time our operations

In [57]:
cores = multiprocessing.cpu_count()
cores

20

In [58]:
w2v_model = Word2Vec(min_count=10,
                     window=5, # The maximum distance between the current and predicted word within a sentence.
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=5,
                     workers=cores-1)

In [59]:
t = time()
w2v_model.build_vocab(sents_tok, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.41 mins


In [60]:
len(w2v_model.wv.vocab) # 134209

132846

In [61]:
list(w2v_model.wv.vocab)[:10]

['ἀρχόμενος',
 'σέο',
 ',',
 'Φοῖβε',
 'κλέα',
 'φωτῶν',
 'μνήσομαι',
 'οἳ',
 'κατὰ',
 'στόμα']

In [62]:
%%time
# previously it took 11.81 mins
w2v_model.train(sents_tok, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

CPU times: user 40min 42s, sys: 13 s, total: 40min 55s
Wall time: 11min 8s


(472980088, 1039951020)

In [63]:
w2v_model

In [64]:
word_vectors = w2v_model.wv

In [65]:
word_vectors

In [66]:
# based on this: https://stackoverflow.com/questions/50466643/in-spacy-how-to-use-your-own-word2vec-model-created-in-gensim
word_vectors.save_word2vec_format("../data/word2vec_win2.txt")

In [181]:
!gzip -k ../data/word2vec_win2.txt